In [1]:
from PIL import Image
import numpy as np
import os

In [2]:
os.chdir('C:\\Users\\monst\\Desktop\\SMT-PRJ\\Hynix_SSL (1)\\data')

In [3]:
npy_list = os.listdir('C:\\Users\\monst\\Desktop\\SMT-PRJ\\Hynix_SSL (1)\\data\\unlabel')
fname_list = os.listdir('C:\\Users\\monst\\Desktop\\SMT-PRJ\\Hynix_SSL (1)\\data\\fname')

In [4]:
data_pre = 'C:\\Users\\monst\\Desktop\\SMT-PRJ\\Hynix_SSL (1)\\data\\unlabel\\unlabel_img_200x200_'
fname_pre = 'C:\\Users\\monst\\Desktop\\SMT-PRJ\\Hynix_SSL (1)\\data\\fname\\unlabel_fname_200x200_'
file_count = len(npy_list)

In [5]:
# unlabel가져와서 합치기
data = [] 
f_data = []
#for i in range(len(npy_list)): # 전체파일로할때
for i in range(1): # 느리니까 일부만 가지고 할때
    data_i = np.load(data_pre+str(i)+'.npy') #이미지 가져오기
    f_data_i = np.load(fname_pre+str(i)+'.npy') #폴더명 가져오기
    if i == 0:
        data = data_i
        f_data = f_data_i
    else:
        data = np.concatenate((data,data_i))
        f_data = np.concatenate((f_data,f_data_i))

In [6]:
data = data # 이미지 - unlabeled
labeled = np.load('C:\\Users\\monst\\Desktop\\SMT-PRJ\\Hynix_SSL (1)\\data\\label\\label_img.npy') # 이미지-labeled
labels  = np.load('C:\\Users\\monst\\Desktop\\SMT-PRJ\\Hynix_SSL (1)\\data\\label\\label_real.npy') # real Y값 - 이미지클래스 


In [7]:
f_label = labels # labeled의 파일경로 원래는 파일경로를 가져와야 하지만, 안가져왔고 앞으로쓸일도없으니 그냥 씀
f_unlabel = f_data # unlabeled의 파일경로

In [8]:
seed    = 0
n_test  = 500
resize=200

In [9]:
np.random.seed(seed) 

In [10]:
u, idx = np.unique(labels, return_inverse=True)
# idx = (np.asarray(new_labels) == 'dust') * 1

test_idx  = np.random.choice(len(idx), n_test, replace=False)
train_idx = np.setdiff1d(np.arange(len(idx)), test_idx)

x_test  = labeled[test_idx, :, :, :]
y_test  = idx[test_idx]
x_train = np.concatenate([labeled[train_idx, :, :, :], data], axis=0)
y_train = idx[train_idx]
f_train = np.concatenate([f_label[train_idx],f_unlabel])
f_test = f_label[test_idx]

In [17]:
import h5py

savep_file = h5py.File('{}x{}_seed{}.h5py'.format(resize, resize, seed), 'w')

dset = savep_file.create_dataset('x_train', np.shape(x_train), dtype='int8')
dset2 = savep_file.create_dataset('y_train', np.shape(y_train), dtype='i')
dset3 = savep_file.create_dataset('x_test', np.shape(x_test), dtype='i')
dset4 = savep_file.create_dataset('y_test', np.shape(y_test), dtype='f')


dtp = h5py.special_dtype(vlen=str)
dset5 = savep_file.create_dataset('f_train', np.shape(f_train), dtype=dtp)
dset6 = savep_file.create_dataset('f_test', np.shape(f_test),dtype=dtp)


dset[:, :, :, :]  = x_train
dset2[:]          = y_train
dset3[:, :, :, :] = x_test
dset4[:]          = y_test
dset5[:]         = f_train
dset6[:]          = f_test

savep_file.close()